In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, pyautogui, re, json
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome()
driver.get("https://lactalis-smartit.onbmc.com/smartit/app/#/")
time.sleep(5)
pyautogui.hotkey("Esc")

In [4]:
driver.find_element(By.NAME, "AUTHENTICATION.LOGIN").send_keys("joaquim.junior.terceiro@br.lactalis.com")
driver.find_element(By.NAME, "AUTHENTICATION.PASSWORD").send_keys("L@ctali_S2024")
driver.find_element(By.NAME, "validateButton").click()

In [5]:
driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div/button').click()
time.sleep(2)
total_chamados = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[4]/h3/span').text
blocos_total = int(total_chamados)//75

In [6]:
def list_chamados(html):
    soup = BeautifulSoup(html, 'html.parser')
    chamados = soup.findAll('span', class_=re.compile('ng-binding ng-scope'))[7:]
    chamados_geral = []
    for chamado in chamados:
        chamados_geral.append(chamado.getText())
    return chamados_geral

def ResetBlocos(blocos_total):
    for bloco in range(0, blocos_total):
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()

In [9]:
chamados_tel = []

for bloco in range(0, blocos_total+1):
    if bloco == 0:
        time.sleep(2)
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
        
ResetBlocos(blocos_total)

In [14]:
dados_chamados = []
for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][8::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            try:
                chave, valor = item.split(' : ', 1)
                dados_dict[chave.strip()] = valor.strip()
            except:
                print(n, item, chamado.split('\n- ')[1:-5])
                break
            
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados.append(dados_dict)
        
df = pd.DataFrame(dados_chamados)

1 Solicitado para :Ivea Carolina Coelho ['Nome do solicitante : Daniele Dziekaniak Pereira', 'Solicitado para :Ivea Carolina Coelho', 'Organização (empresa) : LACTALIS - BR', 'Site (Filial) : BRA-028-ITAJAI', 'Departamento : TECNOLOGIA DA INFORMAÇÃO', 'Diretoria : Finanças', 'Solicitação de periférico de TI : HD SSD', 'Empresa : Lactalis', 'Filial : Itajaí', 'Nome Solicitante : Daniele Dziekaniak Pereira', 'Usuário : Ivea Carolina Coelho', 'Matricula : 547768', 'Centro de Custo : 4700101', 'E-Mail : ivea.coelho@br.lactalis.com', 'Incidente vinculado : INC000001018355', 'Aprovador : Felipe Dias', 'Nome pedido : BR_113 - Solicitação peças e componentes']


In [13]:
dados_chamados = []
for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][8::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            chave, valor = item.split(' : ', 1)
            dados_dict[chave.strip()] = valor.strip()
        
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados.append(dados_dict)
pd.DataFrame(dados_chamados)

ValueError: not enough values to unpack (expected 2, got 1)

In [102]:
chamados_tel

[['3308',
  'INC000001018341',
  '28 de mai de 2024 13:55:29',
  'JOAQUIM MARCELO DOS SANTOS JUNIOR',
  'BRA-026-HUNGRIA',
  'JOAQUIM MARCELO DOS SANTOS JUNIOR',
  'BR_71 - Problema no equipamento',
  'Cancelado',
  '803770 \n- Nome do solicitante : JOAQUIM MARCELO DOS SANTOS JUNIOR\n- Solicitado para : JOAQUIM MARCELO DOS SANTOS JUNIOR\n- Organização (empresa) : LACTALIS - BR\n- Departamento : TECNOLOGIA DA INFORMAÇÃO\n- Site (Filial) : BRA-026-HUNGRIA\n- Diretoria : Tecnologia da Informação\n- Empresa : Lactalis\n- Matrícula : 567777\n- Telefone de contato: : 0\n- Equipamento : Notebook\n- Nome do computador : Microsoft Surface\n- Problema : Outros\n- Descreva seu problema: : Instalação dos aplicativos de segurança Kaspersky e CrowdStrike\n- question_templateName : INC_HD2_Field Services - BR\n- Centro de Custo : 4900028\n- question_Summary : BR_71 - Problema no equipamento\n- question_OperationCategoryTier1 : Hardware / Problema\n- question_serviceType : User Service Restoration\n- 

In [15]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[9::10])
    geral_chamado.extend(tel[8::10])
    status_chamado.extend(tel[7::10])
    tipo_chamado.extend(tel[6::10])
    designado_chamado.extend(tel[5::10])
    filial_chamado.extend(tel[4::10])
    cliente_chamado.extend(tel[3::10])
    criacao_chamado.extend(tel[2::10])
    n_chamado.extend(tel[1::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Designado': designado_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de Criacao': criacao_chamado,
})

df

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao
0,WO0000000346154,MATHEUS LAIDMER DA SILVA,BRA-081-SAO PAULO,MATHEUS LAIDMER DA SILVA,845615 - BR_113 - Solicitação peças e componentes,Pendente,24 de jul de 2024 16:15:38
1,WO0000000345723,Juliano dos Santos Abraao,BRA-038-POUSO ALTO,Daniele Dziekaniak Pereira,844460 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 16:08:27
2,WO0000000345722,Juliano dos Santos Abraao,BRA-038-POUSO ALTO,Daniele Dziekaniak Pereira,844288 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 16:07:36
3,WO0000000345592,FERNANDO REZENDE TAVARES,BRA-084-UBERLANDIA,HUGO FURTADO NEVES,844123 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 10:26:10
4,WO0000000345275,KATIUCI GABRIELE NEIS,BRA-026-HUNGRIA,KATIUCI GABRIELE NEIS,843191 - BR_113 - Solicitação peças e componentes,Pendente,22 de jul de 2024 15:30:58
...,...,...,...,...,...,...,...
115,WO0000000282808,Leandro Coelho,BRA-029-ITUMBIARA,JOAQUIM MARCELO DOS SANTOS JUNIOR,723123 - BR_113 - Solicitação peças e componentes,Pendente,11 de jan de 2024 13:40:48
116,WO0000000278728,Liria Souza,BRA-005-BARRA MANSA,MATHEUS LAIDMER DA SILVA,715512 - BR_113 - Solicitação peças e componentes,Pendente,28 de dez de 2023 09:14:27
117,WO0000000266298,AUGUSTO MULLER VIDAL DE OLIVEIRA,BRA-005-BARRA MANSA,AUGUSTO MULLER VIDAL DE OLIVEIRA,692639 - BR_113 - Solicitação peças e componentes,Pendente,16 de nov de 2023 16:18:59
118,WO0000000261810,Alisson Muniz da Silva,BRA-038-POUSO ALTO,Alisson Muniz da Silva,684128 - BR_113 - Solicitação peças e componentes,Pendente,1 de nov de 2023 10:59:07


In [70]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[8::10])
    geral_chamado.extend(tel[7::10])
    status_chamado.extend(tel[6::10])
    tipo_chamado.extend(tel[5::10])
    designado_chamado.extend(tel[4::10])
    filial_chamado.extend(tel[3::10])
    cliente_chamado.extend(tel[2::10])
    criacao_chamado.extend(tel[1::10])
    n_chamado.extend(tel[::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Designado': designado_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de Criacao': pd.to_datetime(criacao_chamado, format='%d de %b de %Y %H:%M:%S', errors='coerce'),
})
 
df

ValueError: All arrays must be of the same length

In [16]:
df_geral = pd.concat([df, pd.DataFrame(dados_chamados)], axis=1)
df_geral

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao,Nome do solicitante,Solicitado para,Organização (empresa),...,Nome Solicitante,Usuário,Matricula,Centro de Custo,E-Mail,Incidente vinculado,Aprovador,Nome pedido,Modelo equipamento,Informe a peça ou componente a ser adquirido
0,WO0000000346154,MATHEUS LAIDMER DA SILVA,BRA-081-SAO PAULO,MATHEUS LAIDMER DA SILVA,845615 - BR_113 - Solicitação peças e componentes,Pendente,24 de jul de 2024 16:15:38,MATHEUS LAIDMER DA SILVA,MATHEUS LAIDMER DA SILVA,LACTALIS - BR,...,Matheus Laidmer da Silva,Karine Gabriele da Silva,559089,4006461,karine.gabriele@br.lactalis.com,INC000001046415,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
1,WO0000000345723,Juliano dos Santos Abraao,BRA-038-POUSO ALTO,Daniele Dziekaniak Pereira,844460 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 16:08:27,Daniele Dziekaniak Pereira,Daniele Dziekaniak Pereira,LACTALIS - BR,...,Daniele Dziekaniak Pereira,Juliano dos Santos Abraao,041563686,4012203,juliano.abraao@br.lactalis.com,INC000001044927,Felipe Dias,BR_113 - Solicitação peças e componentes,Latitude 3490,NaN
2,WO0000000345722,Juliano dos Santos Abraao,BRA-038-POUSO ALTO,Daniele Dziekaniak Pereira,844288 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 16:07:36,Daniele Dziekaniak Pereira,Juliano dos Santos Abraao,LACTALIS - BR,...,Daniele Dziekaniak Pereira,Juliano dos Santos Abraao,041563686,041563686,juliano.abraao@br.lactalis.com,INC000001044927,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
3,WO0000000345592,FERNANDO REZENDE TAVARES,BRA-084-UBERLANDIA,HUGO FURTADO NEVES,844123 - BR_113 - Solicitação peças e componentes,Pendente,23 de jul de 2024 10:26:10,HUGO FURTADO NEVES,FERNANDO REZENDE TAVARES,LACTALIS - BR,...,Hugo Furtado Neves,FERNANDO REZENDE TAVARES,92186,3014502,NaN,INC000001045542,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
4,WO0000000345275,KATIUCI GABRIELE NEIS,BRA-026-HUNGRIA,KATIUCI GABRIELE NEIS,843191 - BR_113 - Solicitação peças e componentes,Pendente,22 de jul de 2024 15:30:58,KATIUCI GABRIELE NEIS,KATIUCI GABRIELE NEIS,NaN,...,Katiuci Gabriele Neis,FELIPE DE JESUS GONCALES GARCIA,566135,4008503,NaN,INC000001044804,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,WO0000000282808,Leandro Coelho,BRA-029-ITUMBIARA,JOAQUIM MARCELO DOS SANTOS JUNIOR,723123 - BR_113 - Solicitação peças e componentes,Pendente,11 de jan de 2024 13:40:48,JOAQUIM MARCELO DOS SANTOS JUNIOR,JOAQUIM MARCELO DOS SANTOS JUNIOR,LACTALIS - BR,...,Leandro Martins Coelho,Leandro Martins Coelho,423084,4701700,leandro.m.coelho@br.lactalis.com,WO0000000239442,Felipe Dias,BR_113 - Solicitação peças e componentes,Dell\tLatitude 5490 | S/N: 68L4VW2,NaN
116,WO0000000278728,Liria Souza,BRA-005-BARRA MANSA,MATHEUS LAIDMER DA SILVA,715512 - BR_113 - Solicitação peças e componentes,Pendente,28 de dez de 2023 09:14:27,MARLON IURI RIBEIRO BATISTA,MARLON IURI RIBEIRO BATISTA,NaN,...,Liria Souza,Liria Souza,561182,4005805,liria.souza@br.lactalis.com,INC000000956742,Felipe Dias,BR_113 - Solicitação peças e componentes,NaN,NaN
117,WO0000000266298,AUGUSTO MULLER VIDAL DE OLIVEIRA,BRA-005-BARRA MANSA,AUGUSTO MULLER VIDAL DE OLIVEIRA,692639 - BR_113 - Solicitação peças e componentes,Pendente,16 de nov de 2023 16:18:59,AUGUSTO MULLER VIDAL DE OLIVEIRA,AUGUSTO MULLER VIDAL DE OLIVEIRA,LACTALIS - BR,...,Daniela Rodrigues Davet de Freitas,Daniela Rodrigues Davet de Freitas,563087,4005503,daniela.freitas@br.lactalis.com,INC000000913744,Felipe Dias,BR_113 - Solicitação peças e componentes,OptiPlex 3040 DDR3L Desktop,NaN
118,WO0000000261810,Alisson Muniz da Silva,BRA-038-POUSO ALTO,Alisson Muniz da Silva,684128 - BR_113 - Solicitação peças e componentes,Pendente,1 de nov de 2023 10:59:07,Alisson Muniz da Silva,Alisson Muniz da Silva,NaN,...,JOAQUIM MARTINS NETO,JOAQUIM MARTINS NETO,041564950

In [18]:
df_geral.to_excel('Chamados_br_113.xlsx', index=False)